# Torch Regression 


Imvolves the following steps

1. Import necessary libraries
1. Generate synthetic data for Regression case and split the dataset
1. Load data into loader into batches
1. Create a simple model
1. Initialize optimizer and loss function
1. Strat  Training 
1. Draw the plots

# Import necessary libraries

In [ ]:
#import necessary libraries

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset


# Generate synthetic dataset

In [ ]:
#Generate synthetic data for regression
# Generate synthetic dataset (sin(x))
torch.manual_seed(42)
num_samples = 10000
X_reg = torch.unsqueeze(torch.linspace(-2 * np.pi, 2 * np.pi, num_samples), dim=1)
y_reg = torch.sin(X_reg) + 0.1 * torch.randn_like(X_reg)  # adding noise

# Split the dataset

In [ ]:
# Split data into train, val, test sets (70:20:10)
split1 = int(0.7 * len(X_reg))
split2 = int(0.9 * len(X_reg))
X_train_reg, y_train_reg = X_reg[:split1], y_reg[:split1]
X_val_reg, y_val_reg = X_reg[split1:split2], y_reg[split1:split2]
X_test_reg, y_test_reg = X_reg[split2:], y_reg[split2:]

# Load the dataset into batches

In [ ]:
#Create dataloader for batches

# Create DataLoader for training and validation sets
train_dataset_reg = TensorDataset(X_train_reg, y_train_reg)
val_dataset_reg = TensorDataset(X_val_reg, y_val_reg)
test_dataset_reg = TensorDataset(X_test_reg, y_test_reg)

train_loader_reg = DataLoader(train_dataset_reg, batch_size=64, shuffle=True)
val_loader_reg = DataLoader(val_dataset_reg, batch_size=64)
test_loader_reg = DataLoader(test_dataset_reg, batch_size=64)

# Get a simple model

In [ ]:
# Define a simple neural network model for regression
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.linear1 = nn.Linear(1, 5)
        self.linear2 = nn.Linear(5, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x

# Initialize optimizer and loss function

In [ ]:
# Initialize the model, loss function, and optimizer
model_reg = RegressionModel()
criterion_reg = nn.MSELoss()
optimizer_reg = optim.SGD(model_reg.parameters(), lr=0.01)

# Start training

In [ ]:
epochs_reg = 1000
losses_train_reg = []
losses_val_reg = []

for epoch in range(epochs_reg):
    model_reg.train()
    epoch_losses_train = []

    # Training phase
    for inputs, targets in train_loader_reg:
        optimizer_reg.zero_grad()
        outputs_train = model_reg(inputs)
        loss_train = criterion_reg(outputs_train, targets)
        loss_train.backward()
        optimizer_reg.step()
        epoch_losses_train.append(loss_train.item())

    # Calculate mean loss for the epoch
    losses_train_reg.append(np.mean(epoch_losses_train))

    # Evaluation phase (validation)
    model_reg.eval()
    epoch_losses_val = []

    with torch.no_grad():
        for inputs, targets in val_loader_reg:
            outputs_val = model_reg(inputs)
            loss_val = criterion_reg(outputs_val, targets)
            epoch_losses_val.append(loss_val.item())

    # Calculate mean loss for validation set
    losses_val_reg.append(np.mean(epoch_losses_val))

    # Print progress every 50 epochs
    if epoch % 50 == 0:
        print(f'Epoch [{epoch}/{epochs_reg}], Train Loss: {losses_train_reg[-1]:.4f}, Val Loss: {losses_val_reg[-1]:.4f}')


# Plot loss 

In [ ]:
# Plot losses over epochs for regression
plt.figure(figsize=(10, 5))
plt.plot(losses_train_reg, label='Training Loss')
plt.plot(losses_val_reg, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Regression Training and Validation Losses')
plt.legend()
plt.grid(True)
plt.show()

# Plot the learned regression curve
plt.figure(figsize=(10, 5))
plt.scatter(X_reg.numpy(), y_reg.numpy(), color='g', label='Original Data')
plt.plot(X_reg.numpy(), model_reg(X_reg).detach().numpy(), color='r', label='Learned Regression')
plt.xlabel('x')
plt.ylabel('sin(x)')
plt.title('Regression: Original vs Learned Curve')
plt.legend()
plt.grid(True)
plt.show()


